In [ ]:
# Install necessary packages
!pip install --upgrade datasets accelerate
!pip install transformers jiwer evaluate

In [ ]:
# Check the device type, it is highly recommanded to run this code with a gpu device
import torch 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# Load the final model from huggingface

from transformers import WhisperForConditionalGeneration, WhisperProcessor
from transformers import WhisperFeatureExtractor, WhisperTokenizer

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-large-v2")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-large-v2", language="English", task="transcribe")
processor = WhisperProcessor.from_pretrained("Seyfelislem/afripspeech_data_aug")
model = WhisperForConditionalGeneration.from_pretrained("Seyfelislem/afripspeech_data_aug").to(device)
forced_decoder_ids = processor.get_decoder_prompt_ids(language="en", task="transcribe")

In [ ]:
# Load the validation and test splits with streaming mode
from datasets import load_dataset

val_ds  = load_dataset("tobiolatunji/afrispeech-200", "all", split="validation", streaming=True)
test_ds = load_dataset("tobiolatunji/afrispeech-200", "all", split="test", streaming=True)

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [ ]:
# Concatenate both splits in one datset
from datasets import concatenate_datasets

ds = concatenate_datasets([val_ds, test_ds])

In [ ]:
# Generate the audio IDs
IDs = ["ID_" + x["audio_id"].split("/")[0] for x in ds] 

Reading metadata...: 3227it [00:00, 30994.64it/s]
Reading metadata...: 6319it [00:00, 40982.08it/s]


In [ ]:
# Resample all audio files to 16 kHz
from datasets import Audio 

ds = ds.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
# Remove unnecessary columns
ds = ds.remove_columns(["speaker_id","audio_id","path","age_group","gender","accent","domain","country","duration"])
ds = ds.rename_column("transcript","sentence")
ds.features

{'audio': Audio(sampling_rate=16000, mono=True, decode=True, id=None),
 'sentence': Value(dtype='string', id=None)}

In [ ]:
# We Write a function to prepare our data ready for the model inference


def prepare_dataset(batch):
    # load audio data
    audio = batch["audio"]

    # compute input length
    batch["input_length"] = len(batch["audio"])

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["sentence"]).input_ids

    # compute labels length
    batch["labels_length"] = len(batch["labels"])
    return batch

In [ ]:
ds = ds.map(prepare_dataset)

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
# Load the Word Error Rate (WER) metric
import evaluate

metric = evaluate.load("wer")

In [ ]:
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import gc

# Prepare the dataloader of torch 
# Decrease the batch_size if CUDA out of memory error is raised 
eval_dataloader = DataLoader(ds, batch_size=32, collate_fn=data_collator)


In [ ]:
transcripts = []

In [ ]:
for step, batch in enumerate(tqdm(eval_dataloader)):
    with torch.cuda.amp.autocast():
        with torch.no_grad():
            generated_tokens = (
                model.generate(
                    input_features=batch["input_features"].to("cuda"),
                    decoder_input_ids=batch["labels"][:, :4].to("cuda"),
                    max_new_tokens=255,
                )
                .cpu()
                .numpy()
            )

            decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            print(decoded_preds)
            transcripts.extend(decoded_preds)
            print("="*50)
            print(len(transcripts))


0it [00:00, ?it/s]
Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 3227it [00:00, 30926.23it/s]
1it [00:28, 28.11s/it]

['We should be asking ourselves whether we want a heavy-handed approach or a harm reduction approach, she added.\n', 'Tuberculoma is an intracranial mass occurring secondary to dissemination of tuberculosiselsewhere in the body.\n', 'rickets in growing children 2.', 'Patient instructed in how to care for wound and expectations regarding wound closure strips patient and wife verbalized an understanding of information and asked appropriate questions.', 'So what we really really focused on is trying to come up with some new angles and really figure out how this guy deals with the repercussions of his actions in season one.\n', 'Returning hand to the neutral position.', 'Watch patients requiring suicide precautions closely to ensure that they are not cheeking the medication or hiding it in the mouth.', 'Unlike skeletal muscle, cardiac muscle cells do not have a stable resting membrane potential.', 'Increased volume of blood secondary to sodium retention caused by congestive heart failure: 

2it [00:52, 25.84s/it]

['Assess the tightness of the cervical collar. At least one finger should slide under the collar.', 'A drug should not be continued if it is no longer necessary.', 'They aid a strong structural bond between cells and extracellular macromolecules and help to hold tissues together.', 'Before discussing the features of alcoholic liver disease and cirrhosis, a brief outline of methanol metabolism is outlined below and is discussed earlier in Chapter 9 page 239.', 'To counter this, we are encouraging elite and United States tech companies to develop alternative 5G solutions, and we are working alongside them to test these technologies at our military bases, as we speak, the Secretary said.\n', 'With Latin America one of the hardest hit regions in terms of Covid-19 deaths and impact on economic growth, analysts said the outcome in Bolivia could mark a shift toward populism as more elections loom in 2021.\n', 'The lesions of these tissues are embryologically derived from mesoderm, except thos

3it [01:18, 25.95s/it]

...

67it [39:38, 30.47s/it]

['Separation of muscle (nontraumatic), unspecified lower leg. INJECTABLE, INJECTION GEMCITABINE HYDROCHLORIDE, GEMCITABINE HYDROCHLORIDE, EQ 20MG BASE/VIAL. Acute hepatitis B with delta-agent with hepatic coma', 'FEN: Wt3050gms up 10 grams.', 'Intervertebral disc disorders with radiculopathy, lumbosacral region. INJECTABLE, INJECTION CEFADYL, CEFADYL, SODIUM, EQ 4GM BASE/VIAL. Spinal and epidural anesthesia-induced headache during puerperium', 'His blood sugars at 10pm was lower 22 and he was disoriented which resolved with 2 glasses of juice; he did not receive his standing dose of NPH that evening .', "Alcoholic polyneuropathy. INJECTABLE, INJECTION PROCAINAMIDE ACL, PROCAINAMIDE HYDROCHLORIDE, 500MG/ML. Displaced Rolando's fracture, unspecified hand, sequela", 'RESTRAINT PLIFT.', 'Response: HR slightly lower at rest but increases with any activity.', 'may need polycose vs. concentration for cals.', 'desire transfer to Hospital Aquadam ready for level 2 care.', '07 September 2024 2: 

68it [40:04, 29.03s/it]

["NSRST occurs path 80's-110.", 'Attending: MemerioluChief Complaint: Hip pain', 'He triple flexed his bilateral lower extremities to pain .', 'A polyester loop graft is sutured to the distal otic cuff.', 'The real , safe bunker is found later and is used to save 1 , 200 people at the end of season 4 . As for what happened to the second dozen survivors from that bunker ?\n', 'When V02/ABG is ?', 'A 2nd power station was started in Tema New Town and Kisii in 1994 and began making electricity in 1955.\n', 'The consultant Adetula Adebisi and the chief resident agreed the situation with Iyawa Benga could have been complicated by either CHF or dehydration.\n', 'D : 134/5.7/103/27', 'The menstrual history may include peri-menstrual symptoms such as anxiety, uid retention, nervousness, mood fluctuations, food cravings, variations in sexual feelings, and difficulty sleeping.', 'Very small dark brown stool OB+.', 'You had a heart attack and a cardiac catheterization showed a blockage in one of 

69it [40:51, 34.42s/it]

# Post processing

In [ ]:
# remove back to line \n
trs = [x.replace("\n","") for x in transcripts]

In [ ]:
transcripts[0]

'We should be asking ourselves whether we want a heavy-handed approach or a harm reduction approach, she added.\n'

In [ ]:
trs[0]

'We should be asking ourselves whether we want a heavy-handed approach or a harm reduction approach, she added.'

In [ ]:
# Remove special characters 

import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(sentence):
    return re.sub(chars_to_ignore_regex, '', sentence)

In [ ]:
trs_without_puncs = [remove_special_characters(x) for x in trs]

In [ ]:
# Create a pandas dataframe with IDs and transcriptions without punctuations
import pandas as pd 
df = pd.DataFrame({
    "ID" : IDs,
    "transcript" : trs_without_puncs
})

In [ ]:
df.head()

,ID,transcript
0,ID_92d2b94e-3e31-40be-b479-50062e90d1e4,We should be asking ourselves whether we want ...
1,ID_de0b1ee6-46f1-4eb6-a747-476d5b7aace0,Tuberculoma is an intracranial mass occurring ...
2,ID_e80c5101-4c8e-490b-bf7d-04bc453063c0,rickets in growing children 2
3,ID_7de1fa08-ef51-47f1-8f07-d5435d19d4f7,Patient instructed in how to care for wound an...
4,ID_63662c6a-6513-4071-a307-f9bf1b560c70,So what we really really focused on is trying ...


In [ ]:
# save the dataframe to a csv format
df.to_csv("Final_sub_1.csv",index=False)